In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm
import time
import random
import json

Список из 4 шампуней, отзывы на которых возьмём

In [2]:
bad_reviews_links = []
good_reviews_links = []

shampoos = ['https://irecommend.ru/content/shampun-garnier-fructis-obem-i-vosstanovlenie-ekstralegkost-dlya-tonkikh-oslablennykh-volos',
           'https://irecommend.ru/content/shampun-nivea-shampun-ukhod-siyanie-i-obem',
           'https://irecommend.ru/content/shampun-ot-perkhoti-headshoulders-protiv-vypadeniya-volos',
           'https://irecommend.ru/content/shampun-pantene-pro-v-novaya-formula-gladkii-shelk']

In [3]:
bad_reviews_links = ['/content/sploshnoe-vrane-foto-obeshchanii-i-emotsii', '/content/i-gde-zhe-moi-obem-i-vosstanovlenie', '/content/ne-sdelal-volosy-obemnee-ne-vosstanovil-ikh-zato-ulybnul-bredovymi-instrumentalnymi-testami', '/content/obem-moim-tonkim-dlinnym-volosam-deistvitelno-pridal-foto-posle-ispolzovaniya', '/content/ispolzovala-dlya-smyvaniya-maslyanykh-masok-i-prosto-na-kazhdyi-den-moe-mnenie', '/content/legkost-volos-mozhet-byt-vse-zhe-pushistost-eshche-nepromytost', '/content/ekhkak-ya-lyublyu-zapakh-etogo-shampunya-zhal-no-posle-takogo-pridetsya-perestat-im-polzovat', '/content/ne-dlya-moikh-volos-navernoe-khotya-ne-znayu-dazhefoto-posle-mytya', '/content/razocharovanie-528', '/content/vosstanavlivayu-im-poly', '/content/intuitsiya-ne-podvela-1', '/content/snachala-khoroshopotom-uzhasno', '/content/topolinyi-pukh-zhara-iyun-eto-ya-pro-svoi-volosy-posle-mytya-shampunem-nivea-siyanie-i-obem', '/content/shampun-etot-ochen-strannyi-predmet-volosy-chistye-golova-net', '/content/vmesto-brilliantovogo-bleska-i-obema-poluchila-zhestkie-volosy-obem-i-blesk-zabyli-nalit-vo', '/content/dva-minusa-kotorye-zastavili-menya-prekratit-ispolzovanie-dannogo-shampunya', '/content/kraine-nepriyatnyi-effekt-ot-shampunya', '/content/neplokhoi-shampun-no-minus-est-dlya-moikh-volos-ogromnyi', '/content/ziyayushchii-proval-obeshchannogo-siyaniya', '/content/deistvitelno-li-ukhod-i-siyanie', '/content/ot-perkhoti-da-eshche-i-ot-vypadeniya-tak-li-effektiven-pirition-tsinka-kak-nam-pytayutsya-v', '/content/dazhe-nogi-im-myt-ne-stanu-podrobno-rasskazhu-pochemu', '/content/dazhe-i-ne-dumaite-pokupat', '/content/khotite-prekrasnye-issushennye-volosy-i-zhirnye-korni-na-sleduyushchii-dentogdavam-opredelen', '/content/ocherednoe-razocharovanie-veka-chuda-ne-proizoshlo', '/content/net-razmyshleniya-o-marketingovykh-khodakh-odnogo-strannogo-cheloveka', '/content/momentalnyi-effekt-gryaznykh-volos', '/content/budte-ostorozhny-69', '/content/ne-shampun-razocharovanie-0', '/content/ni-ot-perkhoti-ni-ot-vypadeniya-volos-ne-pomogaet', '/content/tak-mnogo-minusov-tak-malo-plyusov', '/content/volosy-stali-vypadat-eshche-bolshe-0', '/content/strannyi-effekt-ubral-perkhot-i-vysushil-i-bez-togo-sukhie-volosy-i-vse-eto-za-odno-primenen', '/content/shampun-isportilsya', '/content/ne-rekomenduyu-1579', '/content/otlichnyi-shampun-432', '/content/vyzyvaet-perkhot-3', '/content/proizvoditeli-opechatalis-ne-protiv-perkhoti-dlya-perkhoti', '/content/chem-dorozhe-tem-kruche-reklama', '/content/ne-ot-vypadeniya-dlya-vypadeniya', '/content/shampun-kak-shampun-5', '/content/uzhasnyi-shampun-96', '/content/ekhpantinnu-ne-sudba-tak-ne-sudbaotzyv-iz-razryadaezhiki-plakalikololisno-prodolzhali-est-ka', '/content/yavno-ne-dlya-chastogo-ispolzovaniya-foto-volos', '/content/ves-lob-v-pryshchakh-foto-shampunya', '/content/pakli-na-golove-byli-pakli-i-ostalis-shampun-ne-sdelal-nichego', '/content/ne-dumala-chto-shelk-vyglyadit-tak', '/content/volosy-neplokho-vyglyadyat-posle-etogo-shampunyano-k-sozhaleniyu-u-nego-uzhasnyi-khimicheski', '/content/vse-chto-ugodno-tolko-ne-shelk', '/content/nachali-lezt-volosy-0', '/content/poyavilas-perkhot-13', '/content/shampun-etoi-serii-ne-vpechatlil-balzam-super', '/content/uzhasnyikak-vse-serii-pantene-pro-v', '/content/moi-verdikt', '/content/okh-i-nadoeli-oni-s-etimi-novymi-formulami', '/content/ostavlyaet-zhelat-luchshego-15', '/content/shelka-chto-ne-vidno', '/content/gladkii-shelk-nesootvetstvie-kakoe', '/content/poyavilas-perkhot-2']

In [4]:
good_reviews_links = ['/content/i-vse-ravno-periodicheski-ruki-tyanutsya-kupit-znamenituyu-zelenuyu-butylochku-ot-garnier-bl', '/content/prekrasnyi-shampun-dlya-tonkikh-i-oslablennykh-volos', '/content/ya-nashla-idealnyi-dlya-sebya-shampun', '/content/poprobuyu-ispolzovat-regulyarnofoto-shampunya-i-kollektsii-garnier', '/content/podoidet-dlya-tonkikh-oslablennykh-volos-foto', '/content/zamechatelnyi-shampun-49', '/content/sukhie-volosy', '/content/blestyashchii-lyubimchik-s-almaznoi-pudroi-kotoromu-moi-volosy-rady-uzhe-3-goda', '/content/myagkii-i-pushistyi-kak-popa-khomyaka', '/content/teper-v-spiske-lyubimchikov-foto-rezultata', '/content/shampun-ot-nivea-vytesnil-s-polki-moego-prezhnego-favorita', '/content/neozhidanno-khoroshii-shampun-1', '/content/pridaet-volosam-shelkovistost', '/content/idealnoe-sredstvo-dlya-moikh-volos-1', '/content/luchshe-lyuksa-3', '/content/normalnyi-shampun-49', '/content/volosy-lezut-klokami-i-vyzyvayut-paniku-shampun-headshoulders-obeshchaet-nam-bystroe-i-nadez', '/content/zamechatelnyi-shampun-pomogaet-ot-perkhoti-foto-volos-v-otzyve', '/content/100-izbavlenie-ot-perkhoti-za-odin-den-bez-vreda-volosam-rekomenduyu-foto', '/content/vpervye-shampun-headshoulders-mne-deistvitelno-pomog-izbavitsya-ot-perkhoti-foto-volos-sosta', '/content/povyshu-obshchii-reiting-etogo-shampunya-mne-ochen-ponravilsya-ne-samyi-plokhoi-shampun-iz-v', '/content/luchshii-shampun-43', '/content/moi-lyubimyi-shampun-47', '/content/odin-iz-lyubimchikov-foto', '/content/mne-ochen-ponravilsya-177', '/content/moe-lichnoe-mnenie-o-shampune-headshoulders', '/content/moi-geroi-1', '/content/pomogaet-348', '/content/khoroshii-shampuns-ochen-priyatnym-aromatom', '/content/otlichnyi-shampun-rekomenduyu-0', '/content/pomogaet-251', '/content/super-414', '/content/odin-iz-nemnogikh-shampunei-kotoryi-mne-idealno-podkhodit', '/content/moi-volosy-vlyubleny-okonchatelno-i-bespovorotno-mnogo-foto-rezultata-ispolzovaniya-neskolki', '/content/moi-volosy-naglyadno-rasskazhut-vam-podoshel-li-im-shampun-pantene-pro-v-gladkii-shelk-foto', '/content/u-menya-svoya-formula-i-volosy-kak-brilliant-foto-dokazatelstva', '/content/mne-bolshe-ne-nuzhen-utyuzhok-foto-do-i-posle', '/content/balzam-gladkii-shelk-chto-poteryano-v-obeme-vospolneno-v-kachestve-volos-foto', '/content/idealno-ochishchaet-volosy-dazhe-ot-masel-dlya-volos-chto-zhe-eshche-delaet-s-volosami', '/content/shampun-ochen-poradoval-foto-vnutri', '/content/udivitelno-chto-tak-malo-otzyvov-na-balzam-pantene-pro-v-gladkii-shelk', '/content/ne-poimu-poddelka-ili-raznye-sostavy-foto', '/content/idealen-dlya-vyushchikhsya-volos-idealen-dlya-menya', '/content/gladkie-pryamye-poslushnye-volosy', '/content/dlya-menya-idealenfoto-volos-posle-primeneniya', '/content/volosy-takie-gladkie-chto-khochetsya-k-nim-prikasatsya', '/content/kachestvo-shampunya-poradovalo-menya-i-moi-volosy', '/content/ochen-dazhe-nichego-258', '/content/luchshii-iz-vsekh-shampunei-chto-ya-ispolzovala', '/content/idealnoe-sredstvo-ya-vlyubilas-v-nego', '/content/nakonets-ya-nashlo-ego', '/content/na-dannyi-moment-best', '/content/otlichnyi-shampun-171', '/content/kupila-pantene-i-ne-razocharovalas', '/content/eto-klassika-2', '/content/otlichnyi-shampun-118', '/content/otlichnyi-shampun-110']

In [5]:
bad_reviews_texts = []
good_reviews_texts = []

Загрузим ранее сохранённое, чтобы не обкачивать заново:

In [6]:
with open('bad_reviews_texts.json', encoding="utf-8") as file:
    bad_reviews_texts = json.load(file)

with open('good_reviews_texts.json', encoding="utf-8") as file:
    good_reviews_texts = json.load(file)

выберем отзывы из разных мест списка, чтобы выборка была более сбалансированной, с разными товарами

In [7]:
bad_check = []
good_check = []
for i in range(0, 56, 8):
    bad_check.append(bad_reviews_texts[i])
    good_check.append(good_reviews_texts[i])
for text in bad_check:
    bad_reviews_texts.remove(text)
for text in good_check:
    good_reviews_texts.remove(text)

In [34]:
print(len(bad_check), len(good_check))
print(len(good_reviews_texts), len(bad_reviews_texts))

7 7
43 43


### теперь чанкер

я хочу выделять "не + прилагательное", "не + глагол" и "наречие + глагол", так как это позволит выделить такие вещи как "не эффективен", "не промывается", "плохо/хорошо расчёсывается" (напоминаю, что мы смотрим отзывы на шампуни, но в целом это объединение должно аналогично работать и для других товаров)

In [9]:
our_tags = ['noun','adj','verb','adv','pron','comp','num','prep','conj','prcl','intj','apro']

In [10]:
nat_dict = {'ADP': 'prep',
            'PROPN': 'noun',
            'AUX': 'verb',
            'DET': 'apro',
            'PART': 'prcl',
            'SCONJ': 'conj',
            'CCONJ': 'conj',
            'PUNCT': 'punct'
           }


def nat_tags(token):
    if 'Degree' in token.feats and token.feats['Degree'] == 'Cmp':
        return 'comp'
    
    tag = token.pos
    if tag.lower() in our_tags:
        return tag.lower()
    
    elif tag in nat_dict:
        new_tag = nat_dict[tag]        
        return new_tag
    
    else:
        return tag

In [11]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [12]:
def chunker(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    nat_tokens = []
    for sent in doc.sents:
        a = sent.morph
        for token in a.tokens:
            tag = nat_tags(token)
            if tag not in ['punct']:
                nat_tokens.append((token.text, tag))
    
    long_token = []
    new_tokens = []
    
    for token in nat_tokens:
        if long_token == []:
            if (token[0] == 'не' or token[1] == 'adv'):
                long_token.append(token)
            else:
                new_tokens.append(token[0])
                #print(token)
        else:
            if long_token[0][1] == 'adv' and token[1] == 'verb':
                long_token.append(token)
                new_tokens.append(long_token[0][0] + ' ' + long_token[1][0])
                #print(long_token)
                long_token = []
            elif long_token[0][0] == 'не' and token[1] in ['verb','adj']:
                long_token.append(token)
                new_tokens.append(long_token[0][0] + ' ' + long_token[1][0])
                #print(long_token)
                long_token = []
            else:
                new_tokens.append(long_token[0][0])
                #print(long_token)
                long_token = []
                if (token[0] == 'не' or token[1] == 'adv'):
                    long_token.append(token)
                else:
                    new_tokens.append(token[0])
                    #print(token)
    
    return new_tokens

Функция, которая выделяет из текста страницы список лемм (уже новых)

In [13]:
def parse_review(raw_text):
    html_content = raw_text.split('сообщить о нарушении')[0]
    html_content = html_content.split('Опубликовано')[-1]
    html_content = html_content.split('Достоинства')[0]
    
    soup = BeautifulSoup(html_content, 'html.parser')
    text = soup.text
    
    tokens = chunker(text)
    return tokens

In [14]:
bad_words = []
good_words = []

In [15]:
for text in tqdm(bad_reviews_texts):
    lemmas = parse_review(text)
    bad_words.extend(lemmas)

100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [00:03<00:00, 11.98it/s]


аналогично с хорошими словами

In [16]:
for text in tqdm(good_reviews_texts):
    lemmas = parse_review(text)
    good_words.extend(lemmas)

100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [00:03<00:00, 14.11it/s]


Теперь множества с только положительными и только отрицательными словами

In [17]:
only_bad = set(bad_words).difference(set(good_words))
only_good = set(good_words).difference(set(bad_words))

фильтрация по количеству вхождений

In [18]:
to_delete = set()
for word in only_bad:
    if bad_words.count(word) < 2:
        to_delete.add(word)

In [19]:
bad = only_bad-to_delete

In [20]:
print(bad)

{'затем', 'продлевает', 'ногти', 'моём', 'не тронутый', 'Лечусь', 'Elisabet', 'не задумываясь', 'скептически', 'случайно глянула', 'понравится', 'радикально остановить', 'химии', 'бутыль', 'р', 'ужасно', 'не восстановил', '69', 'не уменьшает', 'представить', 'Точка', 'распушились', 'лаурилсульфатах', 'замешкалась', 'плакали', 'данных', 'Cocoamphodiacetate', 'красивые', 'любом', 'рассыпчатости', 'журнале', 'не помогает', 'нежели', 'Процедуры', 'керамидами', 'edelweiss', 'начинается', 'желания', 'обычно использую', 'втором', 'АпельсинСпрей', 'Нивея', 'Fragrance', 'рука', 'беседы', 'делают', 'присутствуют', 'sodium', 'не могла', 'не успела', 'версия', 'естественной', 'поэтому наносить', 'выбору', 'Camellia', 'промывают', 'сухих', 'Polyquaternium-30', 'всегда относилась', 'интенсивность', 'Panax', 'Выбор', 'Шикарный', 'Limonum', 'не считается', 'чему', 'внимания', 'Methylpropional', 'не куплю', 'ваших', 'цинк', 'простая', 'Oleth', 'Бигуди', 'усиливает', 'сильная', 'Lemon', 'довольно-таки',

In [21]:
to_delete = set()
for word in only_good:
    if good_words.count(word) < 2:
        to_delete.add(word)

In [22]:
good = only_good-to_delete

In [23]:
print(good)

{'БАЛЬЗАМ', 'Hydrogenated', 'вода', 'подходящий', 'блонд', '30', 'скрипа', 'Кожа', 'опытом', 'PEG-40', 'цветочный', 'перестали', 'естественно', 'Придает', 'обладательница', 'маску', 'пару', 'живые', 'Macadamia', 'Информация', 'небольшой', 'бутылочкой', 'избавление', 'очередь', 'нанести', 'супер', 'Только', 'кончиков', 'Идеальное', 'не люблю', 'Стоит', 'тьфу', 'не сушит', 'здоровый', 'свой', 'необходимо', 'место', 'не жирнит', 'Очищает', 'Polyquaternium-10', 'которым', 'Pro-V', 'задачи', 'возникает', '165', 'Приветствую', 'лимон', 'Никаких', 'енд', 'Мое', 'данным', 'не остается', 'очень нравятся', 'своя', 'уходом', 'нежный', 'маслом', 'искать', 'своей', 'антисептик', 'Замечательный', 'запахом', 'Пантином', 'насчет', 'спустя', 'регулярно использовать', 'смывании', 'Cocamidopropyl', 'чур', 'стороне', 'прежний', 'стабилизатор', 'средству', 'рацион', 'Вывод', 'Зачем', 'радуют', 'те', 'экстрактом', 'лично', 'Elseve', 'Имеет', 'результатом', 'содержится', 'Феном', 'интересно', 'обладательнице

функция, угадывающая, является ли отзыв хорошим

In [24]:
def if_good(lemmas):
    bad_count = 0
    good_count = 0
    for lemma in lemmas:
        if lemma in bad:
            bad_count += 1
        if lemma in good:
            good_count += 1
    print('количество плохих слов:', bad_count,', количество хороших слов:', good_count)
    if bad_count > good_count:
        print('bad')
        return 'bad'
    else:
        print('good')
        return 'good'

In [25]:
tp = 0
tn = 0
fp = 0
fn = 0

In [26]:
for review in bad_check:
    ans = if_good(parse_review(review))
    if ans == 'bad':
        tn += 1
    else:
        fp +=1

количество плохих слов: 13 , количество хороших слов: 2
bad
количество плохих слов: 3 , количество хороших слов: 4
good
количество плохих слов: 4 , количество хороших слов: 4
good
количество плохих слов: 5 , количество хороших слов: 7
good
количество плохих слов: 2 , количество хороших слов: 0
bad
количество плохих слов: 2 , количество хороших слов: 1
bad
количество плохих слов: 6 , количество хороших слов: 0
bad


In [27]:
for review in good_check:
    ans = if_good(parse_review(review))
    if ans == 'good':
        tp += 1
    else:
        fn += 1

количество плохих слов: 5 , количество хороших слов: 7
good
количество плохих слов: 11 , количество хороших слов: 15
good
количество плохих слов: 17 , количество хороших слов: 19
good
количество плохих слов: 7 , количество хороших слов: 4
bad
количество плохих слов: 3 , количество хороших слов: 3
good
количество плохих слов: 4 , количество хороших слов: 10
good
количество плохих слов: 2 , количество хороших слов: 4
good


In [28]:
accuracy = (tn+tp)/(tn+tp+fn+fp)
print(accuracy)

0.7142857142857143


А теперь уберём фильтрацию и посмотрим, что будет

In [29]:
def if_good2(lemmas):
    bad_count = 0
    good_count = 0
    for lemma in lemmas:
        if lemma in only_bad:
            bad_count += 1
        if lemma in only_good:
            good_count += 1
    print('количество плохих слов:', bad_count,', количество хороших слов:', good_count)
    if bad_count > good_count:
        print('bad')
        return 'bad'
    else:
        print('good')
        return 'good'

In [30]:
tp = 0
tn = 0
fp = 0
fn = 0

In [31]:
for review in bad_check:
    ans = if_good2(parse_review(review))
    if ans == 'bad':
        tn += 1
    else:
        fp +=1

количество плохих слов: 22 , количество хороших слов: 11
bad
количество плохих слов: 10 , количество хороших слов: 10
good
количество плохих слов: 12 , количество хороших слов: 10
bad
количество плохих слов: 19 , количество хороших слов: 16
bad
количество плохих слов: 9 , количество хороших слов: 6
bad
количество плохих слов: 6 , количество хороших слов: 14
good
количество плохих слов: 8 , количество хороших слов: 4
bad


In [32]:
for review in good_check:
    ans = if_good2(parse_review(review))
    if ans == 'good':
        tp += 1
    else:
        fn += 1

количество плохих слов: 14 , количество хороших слов: 12
bad
количество плохих слов: 39 , количество хороших слов: 40
good
количество плохих слов: 41 , количество хороших слов: 50
good
количество плохих слов: 10 , количество хороших слов: 10
good
количество плохих слов: 4 , количество хороших слов: 5
good
количество плохих слов: 10 , количество хороших слов: 15
good
количество плохих слов: 10 , количество хороших слов: 12
good


In [33]:
accuracy = (tn+tp)/(tn+tp+fn+fp)
print(accuracy)

0.7857142857142857


Выводы: выборка довольно маленькая, поэтому фильтрация ухудшала результат (удалялись полезные токены, которые просто не успели встретиться больше раз)

В предыдущей версии программы accuracy была 0,57. Сравнение не совсем корректно, так как с прошлого раза код немного изменился (выборка стала больше, возможно, ещё какие-то небольшие изменения), поэтому я посчитала заново, в прошлой версии кода 0,5, в этой (с той же выборкой, просто со словосочетаниями) 0,71. Без фильтрации 0,64 и 0,79 соответственно